In [1]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# Load the CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Flatten the images (convert 32x32x3 images into 1D vectors)
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Normalize the data (important for XGBoost)
scaler = StandardScaler()
X_train_flat = scaler.fit_transform(X_train_flat)
X_test_flat = scaler.transform(X_test_flat)

# Optional: Use a smaller subset of the data to speed up training
X_train_subset, _, y_train_subset, _ = train_test_split(X_train_flat, y_train, test_size=0.9, random_state=42)
X_test_subset, _, y_test_subset, _ = train_test_split(X_test_flat, y_test, test_size=0.9, random_state=42)

# Convert the labels into a 1D array
y_train_subset = y_train_subset.ravel()
y_test_subset = y_test_subset.ravel()

# Create the XGBoost DMatrix objects
dtrain = xgb.DMatrix(X_train_subset, label=y_train_subset)
dtest = xgb.DMatrix(X_test_subset, label=y_test_subset)

# Set XGBoost parameters for multi-class classification
params = {
    'objective': 'multi:softmax',  # Use softmax for multi-class classification
    'num_class': 10,               # 10 classes for CIFAR-10
    'max_depth': 6,                # Maximum depth of the trees
    'eta': 0.3,                    # Learning rate
    'eval_metric': 'mlogloss',      # Multiclass logloss evaluation metric
    'tree_method': 'gpu_hist'      # Use GPU for training
}

# Train the XGBoost model
num_rounds = 50  # Number of boosting rounds
bst = xgb.train(params, dtrain, num_rounds)

2024-09-14 02:06:12.250807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-14 02:06:12.259241: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-14 02:06:12.261699: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-14 02:06:12.268295: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-14 02:06:12.650911: W tensorflow/compiler/tf2

Test accuracy: 0.43


/home/woong/myenv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [02:06:30] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [2]:
y_pred = bst.predict(dtest)

print(y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_test_subset, y_pred)
print(f"Test accuracy: {accuracy:.2f}")

[9. 6. 3. 2. 2. 2. 3. 6. 2. 6. 6. 6. 4. 8. 0. 2. 0. 2. 6. 9. 5. 6. 1. 1.
 4. 1. 2. 9. 9. 5. 9. 0. 2. 8. 2. 0. 6. 0. 2. 7. 0. 2. 7. 6. 9. 7. 5. 0.
 0. 3. 8. 4. 9. 9. 3. 1. 0. 4. 7. 1. 7. 9. 3. 7. 0. 3. 0. 6. 8. 9. 4. 1.
 3. 2. 9. 6. 5. 7. 6. 5. 4. 9. 8. 5. 5. 4. 2. 5. 9. 3. 4. 9. 9. 1. 4. 2.
 2. 5. 1. 7. 8. 7. 9. 5. 4. 9. 9. 3. 5. 9. 3. 9. 6. 5. 4. 6. 7. 6. 5. 0.
 6. 7. 7. 5. 4. 4. 3. 8. 5. 0. 0. 1. 0. 6. 4. 8. 6. 9. 4. 5. 3. 8. 7. 4.
 3. 6. 6. 6. 4. 7. 2. 8. 8. 7. 1. 5. 2. 9. 9. 4. 1. 6. 9. 3. 6. 9. 9. 9.
 8. 4. 7. 6. 0. 0. 9. 6. 6. 9. 2. 0. 7. 0. 1. 2. 5. 4. 1. 4. 9. 8. 2. 7.
 0. 9. 9. 9. 0. 7. 2. 2. 3. 4. 6. 0. 6. 5. 2. 3. 5. 4. 7. 6. 8. 9. 4. 9.
 4. 6. 7. 3. 7. 7. 5. 6. 8. 0. 6. 8. 8. 4. 9. 1. 5. 8. 8. 8. 7. 3. 9. 3.
 9. 4. 2. 0. 2. 6. 0. 0. 0. 1. 7. 0. 2. 3. 0. 1. 1. 5. 9. 0. 4. 5. 6. 5.
 7. 4. 2. 7. 5. 5. 1. 0. 9. 1. 1. 4. 3. 1. 1. 9. 9. 6. 4. 1. 9. 3. 1. 3.
 4. 7. 6. 4. 4. 4. 0. 0. 6. 5. 0. 0. 8. 4. 2. 0. 2. 8. 4. 8. 0. 9. 5. 6.
 9. 9. 8. 2. 7. 0. 0. 0. 0. 5. 4. 4. 4. 4. 9. 7. 2.